# SETUP

In [1]:
!git clone https://github.com/rmihaylov/falcontune.git
!wget https://huggingface.co/TheBloke/falcon-40b-instruct-GPTQ/resolve/main/gptq_model-4bit--1g.safetensors

Cloning into 'falcontune'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 99 (delta 26), reused 33 (delta 22), pack-reused 53
Unpacking objects: 100% (99/99), 56.15 KiB | 1.87 MiB/s, done.
--2023-06-02 02:52:16--  https://huggingface.co/TheBloke/falcon-40b-instruct-GPTQ/resolve/main/gptq_model-4bit--1g.safetensors
Resolving huggingface.co (huggingface.co)... 18.172.170.14, 18.172.170.36, 18.172.170.44, ...
Connecting to huggingface.co (huggingface.co)|18.172.170.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c4/80/c480af4f63a5654be0a115ac717fb2d311418c172f0082e150527ea54a749be9/8c73522c09b190ca0f519a8de47436d0a18048e18c0600dd328d088a1204d9d1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27gptq_model-4bit--1g.safetensors%3B+filename%3D%22gptq_model-4bit--1g.safetensors%22%3B&Expires=1685933537&Pol

In [2]:
# Installation:
!cd falcontune && pip install -r requirements.txt
!cd falcontune && python setup.py install
# !cd falcontune && python setup_cuda.py install  # if cuda, default is triton

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9

In [ ]:
# Restart:
import os; os.kill(os.getpid(), 9)

# GENERATE

In [3]:
!falcontune generate \
    --interactive \
    --model falcon-40b-instruct-4bit \
    --weights gptq_model-4bit--1g.safetensors \
    --max_new_tokens 50 \
    --use_cache \
    --do_sample \
    --prompt "How to prepare pasta?" \
    --backend triton

2023-06-02 02:57:46.402956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/

# FINETUNE

In [ ]:
!wget https://github.com/gururise/AlpacaDataCleaned/raw/main/alpaca_data_cleaned.json

In [ ]:
# Disable wandb:
import os; os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!falcontune finetune \
    --model=falcon-40b-instruct-4bit \
    --weights=./gptq_model-4bit--1g.safetensors \
    --dataset=./alpaca_data_cleaned.json \
    --data_type=alpaca \
    --lora_out_dir=./falcon-40b-instruct-4bit-alpaca/ \
    --mbatch_size=1 \
    --batch_size=2 \
    --epochs=3 \
    --lr=3e-4 \
    --cutoff_len=256 \
    --lora_r=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \
    --save_steps=50 \
    --save_total_limit=3 \
    --logging_steps=5 \
    --target_modules='["query_key_value"]' \
    --backend=triton

# LOAD

In [ ]:
!falcontune generate \
    --interactive \
    --model falcon-40b-instruct-4bit \
    --weights ./gptq_model-4bit--1g.safetensors \
    --lora_apply_dir falcon-40b-instruct-4bit-alpaca/ \
    --max_new_tokens 50 \
    --use_cache \
    --do_sample \
    --instruction "How to prepare pasta?" \
    --backend triton